In [15]:
import sys
sys.path.append("..")  # sube un nivel

import os
import pandas as pd

import geopandas as gpd
from utilities import seleccionar_archivo
from utilities import guardar_como

In [16]:
path_ejecutado = seleccionar_archivo('Seleccione el archivo de puntos', 'shp')
path_ejecutado

'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - SIEMBRA/SIEMBRA CON PRESICION/PARAISO/L9 - TEST3/EJECUTADO-SF3.shp'

In [17]:
path_planificado = seleccionar_archivo('Seleccione el archivo de lineas planificadas', 'shp')
path_planificado

'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - SIEMBRA/SIEMBRA CON PRESICION/PARAISO/L9 - TEST3/PLAN.shp'

In [18]:
# 1. Cargar shapefiles
gdf_puntos = gpd.read_file(path_ejecutado)
gdf_lineas = gpd.read_file(path_planificado)

In [19]:
# 2. Reproyectar ambos a UTM Zona 20S (EPSG:32720) para obtener distancias en metros
gdf_puntos = gdf_puntos.to_crs("EPSG:32720")
gdf_lineas = gdf_lineas.to_crs("EPSG:32720")

In [20]:
len(gdf_puntos)

5787

In [21]:
# 3. Unir todas las líneas en una sola geometría (más eficiente)
lineas_unidas = gdf_lineas.unary_union

In [22]:
# 4. Calcular la distancia desde cada punto a la línea más cercana
gdf_puntos["variacion"] = gdf_puntos.geometry.distance(lineas_unidas)

In [23]:
def clasificar_variacion(valor):
    if valor <= 0.05:
        return 1, "<5 Optimo"
    elif valor <= 0.10:
        return 2, "5-10 Aceptable"
    elif valor <= 0.15:
        return 3, "10-15 Riesgo Moderado"
    else:
        return 4, ">15 Riesgo Alto"

In [24]:
# Aplicar la clasificación a cada fila
gdf_puntos[["cat_num", "categoria"]] = gdf_puntos["variacion"].apply(
    lambda x: pd.Series(clasificar_variacion(x))
)

In [25]:
gdf_puntos = gdf_puntos.rename(columns={
    'Implement1': 'Implmnt1'
})

In [26]:
path_output = guardar_como('Guardar como...', 'shp')
path_output

'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - SIEMBRA/SIEMBRA CON PRESICION/PARAISO/L9 - TEST3/DESVIACION-SF3.shp'

In [27]:
gdf_puntos.to_file(path_output)

In [28]:
# Reemplazar la extensión .shp por .xlsx
ruta_excel = os.path.splitext(path_output)[0] + ".xlsx"
# Exportar la tabla sin geometría
gdf_puntos.drop(columns='geometry').to_excel(ruta_excel, index=False)